In [1]:
import arc
from generalSimulation import GeneralSimulation

from generalDevice import GeneralDevice
from generalChannel import GeneralChannel, default_local_channel_params, default_global_channel_params

import numpy as np

atom = arc.Rubidium()


# n, l, j, mj
ryd1 = (60,0,0.5,0.5)
ryd2 = (60,1,1.5,-0.5)

labels = 'r1','r2'
state_dict = {lb: r for lb, r in zip(labels, [ryd1, ryd2])}
# Need to verify if transitions are valid?


channel_gh = GeneralChannel(
    **default_local_channel_params,
    addressing='Local',
    state1_lbl='g',
    state2_lbl='h', 
)

channel_g1 = GeneralChannel(
    **default_local_channel_params,
    addressing='Local',
    state1_lbl='g',
    state2_lbl=labels[0]
)

channel_g2 = GeneralChannel(
    **default_local_channel_params,
    addressing='Local',
    state1_lbl='g',
    state2_lbl=labels[1]
)

channel_12 = GeneralChannel(
    **default_local_channel_params,
    addressing='Local',
    state1_lbl=labels[0],
    state2_lbl=labels[1]
)

device = GeneralDevice(
    name='ChadoqMulti', 
    dimensions=2, 
    rydberg_states=(ryd1, ryd2),
    state_labels=labels, 
    channel_objects=(
        channel_gh,
        channel_g1,
        channel_g2,
        channel_12
    ),
    channel_ids=('local_gh', 'local_g1', 'local_g2', 'local_l2')
)

In [2]:
import matplotlib.pyplot as plt
from pulser_simulation import Simulation
from pulser.register import Register

from generalSequence import GeneralSequence
from generalSimulation import Simulation

from pulser import Pulse
from pulser.waveforms import RampWaveform, BlackmanWaveform


# distances = np.linspace(6.5, 14, 7)

# r = [1, 0]  # |r>
# rr = np.kron(r, r)  # |rr>
# occup = [np.outer(rr, np.conj(rr))]  # |rr><rr|

duration = 1000  # Typical: ~1 µsec

reg = Register.rectangle(1, 2, spacing=7, prefix="atom")

pi_pulse = Pulse.ConstantDetuning(BlackmanWaveform(duration, np.pi), 0.0, 0.0)


# Pulse Sequence
seq = GeneralSequence(reg, device)
seq.declare_channel('g1', 'local_g1', "atom0")
seq.add(pi_pulse, "g1")
seq.target("atom1", "g1")
seq.add(pi_pulse, "g1")

sim = GeneralSimulation(seq)
# it works????
# Alright, lets test some protocols
# Try some asymmetric blockade


# res = sim.run()  # Returns a SimulationResults instance
# data.append(
#     res.expect(occup)[0]
# )  # Get expectation value for the occupation operator
# print("...Simulation Complete!")

In [7]:
ham = sim._hamiltonian(0)

ham.diag()

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -7.53796818,  0.        ,  0.        ,  0.        ,  0.        ,
        4.46562638])

In [9]:
sim.dim

4